One of the applications of convex optimization in complex variables is phase recovery, a problem in signal processing. The objective is to reconstruct the complex phase of a vector when we are only given the magnitude of some linear measurements.

This problem is orginally a nonconvex optimization problem. Using results from [this paper](https://arxiv.org/abs/1206.0102), the problem can be relaxed to a (complex) semidefinite program (complex SDP).

Given a linear operator $A$ and a vector $b= |Ax|$ of measured amplitudes, define the positive semidefinite hermitian matrix $M = \text{diag}(b) (I - A A^*) \text{diag}(b)$. The problem is:

                minimize < U,M >
                subject to 
                diag(U) = 1
                U in :HermitianSemiDefinite
                
Here the variable $U$ must be hermitian ($U \in \mathbb{H}_n $), and we have a solution to the phase recovery problem if $U = u u^*$ has rank one. Otherwise, the leading singular vector of $U$ can be used to approximate the solution.

Also $Ax = diag(b)u$

In [26]:
using Convex

n = 8
p = 2
A = rand(n,p) + im*randn(n,p)
x = randn(p) + im*randn(p)
b = abs(A*x)
M = diagm(b)*(eye(n)-A*ctranspose(A))*diagm(b)

U = ComplexVariable(n,n)
objective = inner_product(U,M)
c1 = diag(U) == 1 
c2 = U in :SDP
p = minimize(objective,c1,c2)
solve!(p)
U.value

(size(coeff),size(var)) = ((16,16),(16,16))
----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 505
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 129, constraints m = 273
Cones:	primal zero / dual free vars: 137
	sd vars: 136, sd blks: 1
Setup time: 3.64e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf      -inf       inf  1.70e-02 
   100|      inf       inf      -nan      -inf      -inf       inf  4.35e-02 
   200| 4.02e-03  1.56e-04  2.25e-06 -3.09e+02 -3.09e+02  2.08e-17  6.40e-02 


8×8 Array{Complex{Float64},2}:
 0.999999-5.42889e-16im        …           0.998547-0.053849im
         0.0514617+0.998673im              0.105165+0.994453im
         -0.235406+0.971896im             -0.182729+0.983162im
          0.460487-0.887665im              0.412019-0.911173im
          0.992578-0.121577im               0.98459-0.17485im 
          0.559553-0.828793im  …           0.514111-0.857721im
          0.782395+0.622777im              0.814795+0.579742im
          0.998547+0.053849im     0.999999+1.62088e-15im      

In [38]:
# Verify if the rank of U is 1
B, C = eig(U.value);
length([b for b in B if(abs(real(b))>1e-4)])
#Decompose U = uu*
u = C[:,1]
for i in 1:8
        u[i] = u[i]/abs(u[i])
        
end
u

#x = ctranspose(A)*diagm(b)*u
# #verify A*x = diagm(b)*u
#norm(A*x-diagm(b)*u)

8-element Array{Complex{Float64},1}:
 -0.235407-0.971897im
  0.958495-0.28511im 
       1.0+0.0im     
 -0.971122-0.238584im
 -0.351821-0.936067im
 -0.937225-0.348725im
  0.421096-0.907016im
 -0.182729-0.983163im

In [37]:
x

2-element Array{Complex{Float64},1}:
  1.00229+1.01909im  
 -1.20131-0.0487328im

In [ ]:
real()